In [1]:
import pdfplumber
import os
import pandas as pd
from datetime import datetime

In [87]:
class Formateador:
    def __init__(self):
        pass

    def hacer_tabla_global(self):
        self.antibiogramas = self.obtener_datos_todos_los_pacientes()
        df = pd.DataFrame(self.antibiogramas, columns = ['Ingreso', 'Tipo muestra', 'Nº de Cultivo', 'Nº de Orden', 'Rut', 'Nombre', 'Servicio', 'Fecha Firma', 'Resultados'])
                                # 'Amikacina': [],
                                # 'Ampicilina': [],
                                # 'Aztreonam': [],
                                # 'Cloranfenicol': [],
                                # 'Ceftazidima': [],
                                # 'Cefazolina': [],
                                # 'NaN': [],
                                # 'Cefepima': [],
                                # 'Cefotaxima': [],
                                # 'Cefoxitina': [],
                                # 'Ciprofloxacino': [],
                                # 'Clindamicina': [],
                                # 'Colistina': [],
                                # 'Cotrimoxazol': [],
                                # 'Eritromicina': [],
                                # 'Ertapenem': [],
                                # 'Gentamicina': [],
                                # 'Gentamicina 120': [],
                                # 'Imipenem': [],
                                # 'Linezolid': [],
                                # 'Meropenem': [],
                                # 'Nitrofurantoina': [],
                                # 'Penicilina': [],
                                # 'Rifampicina': [],
                                # 'Ampicilina/Sulbactam': [],
                                # 'Sulperazona': [],
                                # 'Tazobactam/Piperacilina': [],
                                # 'Teicoplanina': [],
                                # 'Tigeciclina': [],
                                # 'Tobramicina': [],
                                # 'Levofloxacino': [],
                                # 'Minociclina': [],
                                # 'Daptomicina': [],
                                # 'Vancomicina': []}
        return df

    def formatear_como_df(self, tupla_datos_paciente):
        nombre_paciente, n_orden, rut, fecha_ingreso, fecha_firma, seccion, tipo_muestra, n_cultivo = tupla_datos_paciente[0]
        tabla_sensibilidades = tupla_datos_paciente[1]


        
        pass

    def obtener_datos_todos_los_pacientes(self):
        pacientes = []
        for nombre_archivo in os.listdir():
            if '.pdf' in nombre_archivo:
                paciente = self.obtener_datos_totales_un_paciente(nombre_archivo)
                pacientes.append(paciente)
        

        antibiogramas = [paciente for paciente in pacientes if type(paciente[-1]) == pd.DataFrame]
        return antibiogramas


    def obtener_datos_totales_un_paciente(self, nombre_archivo_paciente):
        datos_personales = self.obtener_datos_personales_paciente(nombre_archivo_paciente)
        resultados_sensibilidad = self.obtener_tabla_antibiograma(nombre_archivo_paciente)

        return [*datos_personales, resultados_sensibilidad]

    # Esta función obteine los datos personales de un paciente, retorna una tupla.
    def obtener_datos_personales_paciente(self, nombre_archivo_paciente):
        with pdfplumber.open(nombre_archivo_paciente) as pdf:
            datos_personales_relevantes = pdf.pages[0].extract_text().split('\n')[3:12]

            nombre_paciente = datos_personales_relevantes[0].split(':')[1][:-10]
            n_orden = datos_personales_relevantes[0].split(':')[-1]
            rut = datos_personales_relevantes[1].split(':')[-1]

            linea_ingreso = datos_personales_relevantes[4].split(' ')
            try:
                fecha_ingreso = datetime.strptime(f'{linea_ingreso[-2]} {linea_ingreso[-1]}', ':%d-%m-%Y %H:%M:%S')
            except ValueError:
                fecha_ingreso = datetime.strptime(f'{linea_ingreso[-2]} {linea_ingreso[-1]}', ':%d/%m/%Y %H:%M:%S')


            linea_firma = datos_personales_relevantes[5].split(' ')
            try:
                fecha_firma = datetime.strptime(f'{linea_firma[-2]} {linea_firma[-1]}', ':%d-%m-%Y %H:%M:%S')
            except ValueError:
                fecha_firma = datetime.strptime(f'{linea_firma[-2]} {linea_firma[-1]}', ':%d/%m/%Y %H:%M:%S')

            seccion = datos_personales_relevantes[6].split(':')[-1]
            tipo_muestra = datos_personales_relevantes[7].split(':')[-1]
            n_cultivo = datos_personales_relevantes[8].split(':', 1)[-1]

            return fecha_ingreso, tipo_muestra, n_cultivo, n_orden, rut, nombre_paciente, seccion, fecha_firma

    # Esta función itera en todo un pdf, para buscar las tablas que sean de antibiograma.
    def obtener_tabla_antibiograma(self, nombre_archivo_paciente):
        with pdfplumber.open(nombre_archivo_paciente) as pdf:
            for pagina in pdf.pages:
                tablas_totales_de_pagina = pagina.extract_table()
                if tablas_totales_de_pagina:
                    for tabla in tablas_totales_de_pagina:
                        tabla = tabla[0].split('\n')
                        if 'ANTIBIOGRAMA' in tabla:
                            df_formateado = self.formatear_tabla_antibiograma(tabla)
                            return df_formateado

                else:
                    return None 

    # Esta función construye la tabla de antibiograma para hacerla un Df. Retorna un Df
    def formatear_tabla_antibiograma(self, lista_tabla):
        separados = list(map(lambda x: x.split(' '), lista_tabla))
        nombre_tabla, headers = separados[0], separados[1]
        numero_cepas = [dato for dato in headers if dato.isnumeric()]

        columnas_headers = ['ANTIBIOTICOS']
        for numero_cepa in numero_cepas:
            columnas_headers.append(f'Cepa {numero_cepa}')
            columnas_headers.append(f'CIM')
        

        datos_susceptibilidad = separados[2:]
        for dato in datos_susceptibilidad:
            espacios_a_llenar_en_linea = len(columnas_headers) - len(dato)
            dato += [None] * espacios_a_llenar_en_linea

        df = pd.DataFrame(datos_susceptibilidad, columns = columnas_headers)
        return df
    


In [88]:
formateador = Formateador()
df = formateador.hacer_tabla_global()

In [90]:
prueba = df['Resultados'][3].copy()

In [108]:
prueba = prueba.set_index('ANTIBIOTICOS')

In [100]:
lista_datos_paciente = ['Ingreso', 'Tipo muestra', 'Nº de Cultivo', 'Nº de Orden', 'Rut', 'Nombre', 'Servicio', 'Fecha Firma', 'Microorganismo']
lista_farmacos_totales = ['Amikacina','Ampicilina','Aztreonam','Cloranfenicol','Ceftazidima','Cefazolina','NaN','Cefepima','Cefotaxima','Cefoxitina','Ciprofloxacino',
                        'Clindamicina',
                        'Colistina',
                        'Cotrimoxazol',
                        'Eritromicina',
                        'Ertapenem',
                        'Gentamicina',
                        'Gentamicina 120',
                        'Imipenem',
                        'Linezolid',
                        'Meropenem',
                        'Nitrofurantoina',
                        'Penicilina',
                        'Rifampicina',
                        'Ampicilina/Sulbactam',
                        'Sulperazona',
                        'Tazobactam/Piperacilina',
                        'Teicoplanina',
                        'Tigeciclina',
                        'Tobramicina',
                        'Levofloxacino',
                        'Minociclina',
                        'Daptomicina',
                        'Vancomicina']


columnas_totales = lista_datos_paciente + lista_farmacos_totales
df = pd.DataFrame(columns = columnas_totales)
df
                                

,Ingreso,Tipo muestra,Nº de Cultivo,Nº de Orden,Rut,Nombre,Servicio,Fecha Firma,Microorganismo,Amikacina,...,Ampicilina/Sulbactam,Sulperazona,Tazobactam/Piperacilina,Teicoplanina,Tigeciclina,Tobramicina,Levofloxacino,Minociclina,Daptomicina,Vancomicina


In [113]:
diccionarios_farmacos_limpios = {key:None for key in lista_farmacos_totales}

for resultado in prueba.index:
    diccionarios_farmacos_limpios[resultado] = prueba['Cepa 1'][resultado]

diccionarios_farmacos_limpios

{'Amikacina': 'Resistente',
 'Ampicilina': None,
 'Aztreonam': 'Resistente',
 'Cloranfenicol': None,
 'Ceftazidima': 'Resistente',
 'Cefazolina': None,
 'NaN': None,
 'Cefepima': None,
 'Cefotaxima': 'Resistente',
 'Cefoxitina': None,
 'Ciprofloxacino': 'Resistente',
 'Clindamicina': None,
 'Colistina': 'Sensible',
 'Cotrimoxazol': None,
 'Eritromicina': None,
 'Ertapenem': 'Resistente',
 'Gentamicina': 'Resistente',
 'Gentamicina 120': None,
 'Imipenem': 'Resistente',
 'Linezolid': None,
 'Meropenem': 'Resistente',
 'Nitrofurantoina': None,
 'Penicilina': None,
 'Rifampicina': None,
 'Ampicilina/Sulbactam': 'Resistente',
 'Sulperazona': None,
 'Tazobactam/Piperacilina': None,
 'Teicoplanina': None,
 'Tigeciclina': 'Sensible',
 'Tobramicina': None,
 'Levofloxacino': None,
 'Minociclina': None,
 'Daptomicina': None,
 'Vancomicina': None,
 'Ceftolozano/Tazobactam': 'Resistente',
 'Piperacilina/Tazobactam': 'Resistente',
 'Ceftazidima/avibactam': 'Resistente'}

In [109]:
prueba

,Cepa 1,CIM
ANTIBIOTICOS,,
Amikacina,Resistente,>32
Aztreonam,Resistente,>16
Ceftazidima,Resistente,>16
Ciprofloxacino,Resistente,>2
Colistina,Sensible,None
Ceftolozano/Tazobactam,Resistente,>8/4
Cefotaxima,Resistente,>4
Ertapenem,Resistente,>1
Gentamicina,Resistente,>8
